# 1 Dataset Creation

Author: Sandra Godinho Silva \
Most updated version: 0.2 from 07/09/2020

In [1]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns

## 1 - Create input dataset
### 1.1 Download Genomes

#### **Some notes:**
**Download done on 7 September 2020:** \
Taxon **Flavobacteriia** (txid 117743) \
Latest GenBank: 3,493 genomes\
Latest RefSeq: 2,093 genomes

**Download GenBank in fasta format files** \
RefSeq genomes are a subset of the available GenBank genomes, so the number of RefSeq genomes is lower. \
Our goal is to get the maximum number of genomes. The filtering step for quality and taxonomy will be done afterwards. \
As we are going to use the genomes in an annotated format (fasta) and the main difference between GenBank and RefSeq is the annotation pipeline, downloading GenBank assemblies fits our needs. \
https://www.ncbi.nlm.nih.gov/genome/doc/ftpfaq/

### 1.2 Create dataset table

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 1000)

In [3]:
gbk = pd.read_csv("gbk_files.txt")
gbk = gbk["gbk_files.txt"].str.replace(".gbk", "")
gbk.head()

0    GCA_000016645.1
1    GCA_000023285.1
2    GCA_000023465.1
3    GCA_000023725.1
4    GCA_000024125.1
Name: gbk_files.txt, dtype: object

In [4]:
assemblies = pd.read_csv("assembly_ids.csv", header=None)
assemblies = assemblies.rename(columns={0:"I"})
assemblies.head()

,I
0,GCA_000016645.1
1,GCA_000023285.1
2,GCA_000023465.1
3,GCA_000023725.1
4,GCA_000024125.1


In [5]:
assemblies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2680 entries, 0 to 2679
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   I       2680 non-null   object
dtypes: object(1)
memory usage: 21.1+ KB


In [6]:
merge = pd.merge(assemblies, gbk, how="outer", left_on="I", right_on="gbk_files.txt", indicator="merge")
merge[merge["merge"] == "left_only"]["I"].to_csv("Missing_to_prokka.txt", index=False)

In [7]:
faa = pd.read_csv("faa.txt")
faa = faa["faa.txt"].str.replace(".faa", "")
faa.head()
merge = pd.merge(assemblies, faa, how="outer", left_on="I", right_on="faa.txt", indicator="merge")
merge[merge["merge"] == "left_only"]#["I"]#.to_csv("Missing_to_prokka.txt", index=False)

,I,faa.txt,merge


In [8]:
gbk = pd.read_csv("gbk.txt")
gbk = gbk["gbk.txt"].str.replace(".gbk", "")
gbk.head()
merge = pd.merge(assemblies, gbk, how="outer", left_on="I", right_on="gbk.txt", indicator="merge")
merge[merge["merge"] == "left_only"]#["I"]#.to_csv("Missing_to_prokka.txt", index=False)

,I,gbk.txt,merge


## 2 - Run CheckM - Get genome quality
https://github.com/Ecogenomics/CheckM/wiki

### 2.1 Run CheckM

### 2.2. Upload CheckM results

In [9]:
checkm = pd.read_csv("output_checkm.tsv", sep='\t')

### 2.3 Calculation of the Quality score

In [10]:
checkm["Quality_score"] = checkm["Completeness"] - 5*checkm["Contamination"]

In [11]:
checkm = checkm.rename(columns = {'Strain heterogeneity': 'Strain_heterogeneity', 'Bin Id': 'Bin_Id'})

In [12]:
checkm = checkm[["Bin_Id", "Marker lineage", "Completeness", "Contamination", "Strain_heterogeneity", "Quality_score"]]

In [13]:
checkm.head()

,Bin_Id,Marker lineage,Completeness,Contamination,Strain_heterogeneity,Quality_score
0,GCA_000016645.1_ASM1664v1_genomic,f__Flavobacteriaceae (UID2817),99.65,0.14,0.0,98.95
1,GCA_000017525.1_ASM1752v1_genomic,k__Bacteria (UID2565),44.01,0.00,0.0,44.01
2,GCA_000022605.2_ASM2260v1_genomic,k__Bacteria (UID2565),98.82,0.00,0.0,98.82
3,GCA_000022945.1_ASM2294v1_genomic,k__Bacteria (UID2565),49.30,0.00,0.0,49.30
4,GCA_000023285.1_ASM2328v1_genomic,p__Bacteroidetes (UID2605),100.00,0.00,0.0,100.00


## 3 - Run GTDBtk - Taxonomy

### 3.1 Run GTDB-Tk v.1.3.0
GTDB release 95.0 (GTDB R05-RS95) https://gtdb.ecogenomic.org/ \
https://ecogenomics.github.io/GTDBTk/

### 3.2 Upload GTDB-Tk results

In [14]:
#GenBank: 3,493 genomes
gtdb = pd.read_csv("gtdbtk.bac120.summary.tsv", sep="\t")

In [15]:
gtdb[["1", "Domain", "Phyla", "Class", "Order", "Family", "Genus", "Species"]] = gtdb.classification.str.split(".__", expand=True)
gtdb[["Domain", "Phyla", "Class", "Order", "Family", "Genus"]] = gtdb[["Domain", "Phyla", "Class", "Order", "Family", "Genus"]].replace(";", "", regex=True)

In [16]:
gtdb = gtdb[["user_genome","classification", "Domain", "Phyla", "Class", "Order", "Family", "Genus", "Species"]]

In [17]:
gtdb.head()

,user_genome,classification,Domain,Phyla,Class,Order,Family,Genus,Species
0,GCA_008011715.1_ASM801171v1_genomic,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Pseudoduganella,Pseudoduganella sp003293715
1,GCA_004151235.1_ASM415123v1_genomic,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Flavobacterium anhuiense_A
2,GCA_900101855.1_IMG-taxon_2596583647_annotated...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Flavobacterium anhuiense
3,GCA_001705175.1_ASM170517v1_genomic,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Flavobacterium anhuiense
4,GCA_000278705.1_ASM27870v1_genomic,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Flavobacterium anhuiense


In [18]:
len(gtdb)

3482

In [56]:
len(gtdb[gtdb["Family"] == "Flavobacteriaceae"]) + len(gtdb[gtdb["Family"] == "Weeksellaceae"])

2826

### 3.3 Solve nomenclature conflicts

**Family**

In [19]:
gtdb = gtdb.astype(str)

In [20]:
filt = (gtdb["Family"] == "Flavobacteriaceae") & (gtdb.Genus =="")
gtdb.loc[filt,"Genus"] = "Uncl. Flavobacteriaceae"

In [21]:
filt = (gtdb["Family"] == "Weeksellaceae") & (gtdb.Genus =="")
gtdb.loc[filt,"Genus"] = "Uncl. Weeksellaceae"

**Genus**

In [22]:
l = gtdb.Genus.unique()

In [23]:
for x in l:
    filt = (gtdb["Genus"] == str(x)) & (gtdb.Species == "")
    s = ' '.join([str(x), "sp."])
    gtdb.loc[filt,"Species"] = s

## 4 - Get more information of the dataset

### 4.1 - Run BBtools
BBtools: https://jgi.doe.gov/data-and-tools/bbtools/

### 4.2 Upload BBtools results

In [24]:
bbtools = pd.read_csv("bbtools.txt", sep="\t")

In [25]:
bbtools["file"] = bbtools["filename"].str.split("/", expand=True)[8].str.replace(".fna","")
bbtools = bbtools.drop(columns=["filename"])

In [26]:
bbtools.head()

,n_scaffolds,n_contigs,scaf_bp,contig_bp,gap_pct,scaf_N50,scaf_L50,ctg_N50,ctg_L50,scaf_N90,scaf_L90,ctg_N90,ctg_L90,scaf_max,ctg_max,scaf_n_gt50K,scaf_pct_gt50K,gc_avg,gc_std,file
0,1,1,6096872,6096872,0.0,1,6096872,1,6096872,1,6096872,1,6096872,6096872,6096872,1,100.000,0.34113,0.0000,GCA_000016645.1_ASM1664v1_genomic
1,1,1,245530,245530,0.0,1,245530,1,245530,1,245530,1,245530,245530,245530,1,100.000,0.22441,0.0000,GCA_000017525.1_ASM1752v1_genomic
2,2,2,640935,640935,0.0,1,636850,1,636850,1,636850,1,636850,636850,636850,1,99.363,0.27159,0.0135,GCA_000022605.2_ASM2260v1_genomic
3,1,1,276984,276984,0.0,1,276984,1,276984,1,276984,1,276984,276984,276984,1,100.000,0.22595,0.0000,GCA_000022945.1_ASM2294v1_genomic
4,1,1,2612925,2612925,0.0,1,2612925,1,2612925,1,2612925,1,2612925,2612925,2612925,1,100.000,0.39588,0.0000,GCA_000023285.1_ASM2328v1_genomic


## 5 - Create final table

### 5.1 Merge

In [27]:
dataset = pd.merge(checkm, gtdb, how = "left", left_on="Bin_Id", right_on="user_genome", indicator="merge")

In [28]:
dataset = pd.merge(dataset, bbtools, how = "left", left_on="Bin_Id", right_on="file", indicator="merge2")

In [29]:
dataset["merge"].value_counts()

both          3482
left_only       10
right_only       0
Name: merge, dtype: int64

In [30]:
dataset["merge2"].value_counts()

both          3492
right_only       0
left_only        0
Name: merge2, dtype: int64

In [31]:
dataset = dataset.drop(columns=["merge", "merge2", "user_genome", "file"])

In [32]:
dataset.head()

,Bin_Id,Marker lineage,Completeness,Contamination,Strain_heterogeneity,Quality_score,classification,Domain,Phyla,Class,Order,Family,Genus,Species,n_scaffolds,n_contigs,scaf_bp,contig_bp,gap_pct,scaf_N50,scaf_L50,ctg_N50,ctg_L50,scaf_N90,scaf_L90,ctg_N90,ctg_L90,scaf_max,ctg_max,scaf_n_gt50K,scaf_pct_gt50K,gc_avg,gc_std
0,GCA_000016645.1_ASM1664v1_genomic,f__Flavobacteriaceae (UID2817),99.65,0.14,0.0,98.95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Flavobacterium johnsoniae,1,1,6096872,6096872,0.0,1,6096872,1,6096872,1,6096872,1,6096872,6096872,6096872,1,100.000,0.34113,0.0000
1,GCA_000017525.1_ASM1752v1_genomic,k__Bacteria (UID2565),44.01,0.00,0.0,44.01,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Blattabacteriaceae,Sulcia,Sulcia sp.,1,1,245530,245530,0.0,1,245530,1,245530,1,245530,1,245530,245530,245530,1,100.000,0.22441,0.0000
2,GCA_000022605.2_ASM2260v1_genomic,k__Bacteria (UID2565),98.82,0.00,0.0,98.82,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Blattabacteriaceae,Blattabacterium,Blattabacterium sp000022605,2,2,640935,640935,0.0,1,636850,1,636850,1,636850,1,636850,636850,636850,1,99.363,0.27159,0.0135
3,GCA_000022945.1_ASM2294v1_genomic,k__Bacteria (UID2565),49.30,0.00,0.0,49.30,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Blattabacteriaceae,Sulcia,Sulcia muelleri,1,1,276984,276984,0.0,1,276984,1,276984,1,276984,1,276984,276984,276984,1,100.000,0.22595,0.0000
4,GCA_000023285.1_ASM2328v1_genomic,p__Bacteroidetes (UID2605),100.00,0.00,0.0,100.00,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Capnocytophaga,Capnocytophaga ochracea,1,1,2612925,2612925,0.0,1,2612925,1,2612925,1,2612925,1,2612925,2612925,2612925,1,100.000,0.39588,0.0000


In [33]:
l1 = dataset["Bin_Id"].str.split("_", expand=True).iloc[:,0:2]
dataset["Genome_ID"]=l1.apply(lambda two: '_'.join(two.dropna().astype(str).values), axis=1)
l2 = dataset.columns.tolist() # list the columns in the df
l2.insert(0, l2.pop(l2.index('Genome_ID'))) # Assign new position for column Bin
dataset = dataset.reindex(columns= l2)
dataset.head()

,Genome_ID,Bin_Id,Marker lineage,Completeness,Contamination,Strain_heterogeneity,Quality_score,classification,Domain,Phyla,Class,Order,Family,Genus,Species,n_scaffolds,n_contigs,scaf_bp,contig_bp,gap_pct,scaf_N50,scaf_L50,ctg_N50,ctg_L50,scaf_N90,scaf_L90,ctg_N90,ctg_L90,scaf_max,ctg_max,scaf_n_gt50K,scaf_pct_gt50K,gc_avg,gc_std
0,GCA_000016645.1,GCA_000016645.1_ASM1664v1_genomic,f__Flavobacteriaceae (UID2817),99.65,0.14,0.0,98.95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Flavobacterium johnsoniae,1,1,6096872,6096872,0.0,1,6096872,1,6096872,1,6096872,1,6096872,6096872,6096872,1,100.000,0.34113,0.0000
1,GCA_000017525.1,GCA_000017525.1_ASM1752v1_genomic,k__Bacteria (UID2565),44.01,0.00,0.0,44.01,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Blattabacteriaceae,Sulcia,Sulcia sp.,1,1,245530,245530,0.0,1,245530,1,245530,1,245530,1,245530,245530,245530,1,100.000,0.22441,0.0000
2,GCA_000022605.2,GCA_000022605.2_ASM2260v1_genomic,k__Bacteria (UID2565),98.82,0.00,0.0,98.82,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Blattabacteriaceae,Blattabacterium,Blattabacterium sp000022605,2,2,640935,640935,0.0,1,636850,1,636850,1,636850,1,636850,636850,636850,1,99.363,0.27159,0.0135
3,GCA_000022945.1,GCA_000022945.1_ASM2294v1_genomic,k__Bacteria (UID2565),49.30,0.00,0.0,49.30,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Blattabacteriaceae,Sulcia,Sulcia muelleri,1,1,276984,276984,0.0,1,276984,1,276984,1,276984,1,276984,276984,276984,1,100.000,0.22595,0.0000
4,GCA_000023285.1,GCA_000023285.1_ASM2328v1_genomic,p__Bacteroidetes (UID2605),100.00,0.00,0.0,100.00,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Capnocytophaga,Capnocytophaga ochracea,1,1,2612925,2612925,0.0,1,2612925,1,2612925,1,2612925,1,2612925,2612925,2612925,1,100.000,0.39588,0.0000


### 5.2 Classification of genomes
High - Medium - Low

In [34]:
def classifier(row):
    if (row["Quality_score"] > 50) and (row["Completeness"] >90) and (row["Contamination"] <5)  and (row["Strain_heterogeneity"] <10) and (row["n_contigs"]<500): #and (row["scaf_N50"] > 10)
        return "High"
    elif (row["Quality_score"] > 50) and (row["Completeness"] >80) and (row["Contamination"] <10)  and (row["Strain_heterogeneity"] <50):
        return "Medium"
    elif  (row["Completeness"] >50) and (row["Contamination"] <10) and  (row["Quality_score"] > 50):
        return "Low"
    else:
        return "Error"

In [35]:
dataset["Classification_quality"] = dataset.apply(classifier, axis=1)

In [36]:
dataset["Classification_quality"].value_counts()

High      2150
Low        651
Medium     436
Error      255
Name: Classification_quality, dtype: int64

### 5.3 Remove genomes below minimum quality

In [37]:
dataset = dataset[~(dataset["Classification_quality"] =="Error")]

### 5.4 Only include Flavobacteriaceae and Weeksleaceae families

In [38]:
dataset = dataset[dataset.Family.str.contains("Flavobacteriaceae", na=False) | dataset.Family.str.contains("Weeksellaceae", na=False)]

In [39]:
dataset["Family"].value_counts()

Flavobacteriaceae    1924
Weeksellaceae         757
Name: Family, dtype: int64

In [40]:
dataset.to_csv("Dataset.csv", index=False)

In [41]:
len(dataset)

2681

## 6 - Table with Genus that have at least x genomes

In [42]:
dataset.groupby('Genus')["Genus"].nunique().count()
print(dataset.groupby('Genus')["Genus"].nunique().count())
len(dataset)

175


2681

In [43]:
dataset[dataset["Family"]=="Flavobacteriaceae"].groupby('Genus')["Genus"].nunique().count()

150

In [44]:
dataset.groupby('Genus')["Genus"].value_counts()

Genus              Genus            
AU392              AU392                 2
Aequorivita        Aequorivita          42
Algibacter         Algibacter            9
Algibacter_A       Algibacter_A          1
Algibacter_B       Algibacter_B          5
                                        ..
YIM-102668         YIM-102668            2
Zeaxanthinibacter  Zeaxanthinibacter     3
Zhouia             Zhouia                2
Zobellia           Zobellia              6
Zunongwangia       Zunongwangia         13
Name: Genus, Length: 175, dtype: int64

In [45]:
dataset_reduced_8 = dataset.groupby(['Genus']).filter(lambda x:x['Genus'].count()>7)
print(dataset_reduced_8.groupby('Family')["Family"].value_counts())
print(dataset_reduced_8.groupby('Genus')["Genus"].nunique().count())
print(dataset_reduced_8.groupby('Family')["Genus"].nunique().count())
len(dataset_reduced_8)

Family             Family           
Flavobacteriaceae  Flavobacteriaceae    1650
Weeksellaceae      Weeksellaceae         710
Name: Family, dtype: int64
60
2


2360

In [46]:
dataset_reduced_8.groupby('Genus')["Genus"].value_counts()

Genus                    Genus                  
Aequorivita              Aequorivita                 42
Algibacter               Algibacter                   9
Apibacter                Apibacter                   21
Aquimarina               Aquimarina                  39
Arenibacter              Arenibacter                 22
BACL21                   BACL21                      15
CG1-02-35-72             CG1-02-35-72                 8
CG2-30-34-30             CG2-30-34-30                 8
Capnocytophaga           Capnocytophaga              75
Cellulophaga             Cellulophaga                21
Chryseobacterium         Chryseobacterium           294
Croceivirga              Croceivirga                  8
Dokdonia                 Dokdonia                    11
Elizabethkingia          Elizabethkingia            193
Empedobacter             Empedobacter                44
Epilithonimonas          Epilithonimonas             31
Euzebyella               Euzebyella                   9

In [47]:
dataset_reduced_8.to_csv("Dataset_reduced.csv", index=False)

In [48]:
dataset_reduced_8["Genome_ID"].to_csv("Dataset_reduced_8.csv")

In [49]:
dataset_reduced_8.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2360 entries, 0 to 3491
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Genome_ID               2360 non-null   object 
 1   Bin_Id                  2360 non-null   object 
 2   Marker lineage          2360 non-null   object 
 3   Completeness            2360 non-null   float64
 4   Contamination           2360 non-null   float64
 5   Strain_heterogeneity    2360 non-null   float64
 6   Quality_score           2360 non-null   float64
 7   classification          2360 non-null   object 
 8   Domain                  2360 non-null   object 
 9   Phyla                   2360 non-null   object 
 10  Class                   2360 non-null   object 
 11  Order                   2360 non-null   object 
 12  Family                  2360 non-null   object 
 13  Genus                   2360 non-null   object 
 14  Species                 2360 non-null   

# 7 - Statistics

In [51]:
with pd.ExcelWriter('Counts.xlsx') as writer:  
    dataset["Species"].value_counts().to_excel(writer, sheet_name="Species_counts")
    dataset["Genus"].value_counts().to_excel(writer, sheet_name="Genus_counts")
    dataset["Family"].value_counts().to_excel(writer, sheet_name="Family_counts")

In [52]:
statistics = dataset.describe().loc[["mean", "std", "min", "25%", "50%", "75%", "max"]].round(2)
statistics

,Completeness,Contamination,Strain_heterogeneity,Quality_score,n_scaffolds,n_contigs,scaf_bp,contig_bp,gap_pct,scaf_N50,scaf_L50,ctg_N50,ctg_L50,scaf_N90,scaf_L90,ctg_N90,ctg_L90,scaf_max,ctg_max,scaf_n_gt50K,scaf_pct_gt50K,gc_avg,gc_std
mean,96.07,0.84,9.98,91.85,81.36,105.55,3578576.36,3574200.99,0.16,14.38,953412.35,17.61,877064.74,44.14,732178.26,55.14,676630.70,1176042.33,1094902.15,11.48,78.22,0.36,0.03
std,8.50,1.18,23.27,11.39,132.49,216.76,1102195.71,1103229.29,0.66,30.13,1430573.91,42.49,1394955.02,91.05,1445028.39,134.23,1408055.72,1364424.27,1334361.10,9.13,31.98,0.04,0.02
min,50.31,0.00,0.00,50.01,1.00,1.00,569462.00,569462.00,0.00,1.00,2659.00,1.00,1884.00,1.00,545.00,1.00,518.00,11420.00,9123.00,0.00,0.00,0.28,0.00
25%,98.35,0.09,0.00,91.32,14.00,17.00,2805039.00,2800951.00,0.00,2.00,83912.00,2.00,76644.00,6.00,21204.00,7.00,19917.00,233227.00,218957.00,2.00,71.32,0.33,0.01
50%,99.52,0.49,0.00,96.37,39.00,44.00,3545181.00,3542912.00,0.00,5.00,294681.00,5.00,259832.00,14.00,85343.00,15.00,75351.00,634411.00,578043.00,11.00,94.89,0.35,0.03
75%,100.00,1.08,0.00,98.75,90.00,101.00,4276845.00,4275762.00,0.02,12.00,821621.00,13.00,679776.00,39.00,235479.00,42.00,188725.00,1481894.00,1222563.00,18.00,99.15,0.37,0.04
max,100.00,9.68,100.00,100.00,1512.00,5612.00,7041311.00,7041311.00,11.48,418.00,6653812.00,1103.00,6653812.00,1182.00,6653812.00,3546.00,6653812.00,6653812.00,6653812.00,54.00,100.00,0.55,0.17


In [53]:
statistics_classification = dataset.groupby("Classification_quality").describe()
statistics_classification2 = statistics_classification.T.round(2)
col = ["High", "Medium", "Low"]
statistics_classification2 = statistics_classification2[col]
statistics_classification = statistics_classification2.loc(axis=0)[:, ["mean", "min", "max"]]
statistics_classification.T

Completeness               Contamination             \
                               mean    min    max          mean  min   max   
Classification_quality                                                       
High                          99.27  90.14  100.0          0.50  0.0  4.16   
Medium                        93.03  80.03  100.0          1.93  0.0  9.56   
Low                           83.07  50.31  100.0          1.68  0.0  9.68   

                       Strain_heterogeneity              Quality_score         \
                                       mean  min     max          mean    min   
Classification_quality                                                          
High                                   0.07  0.0    9.09         96.79  71.57   
Medium                                18.88  0.0   47.22         83.39  50.14   
Low                                   50.60  0.0  100.00         74.69  50.01   

                              n_scaffolds              n_contigs               \
                          max        mean  min     max      mean  min     max   
Classification_quality                                                          
High                    100.0       47.00  1.0   434.0     52.56  1.0   465.0   
Medium                  100.0      171.02  1.0  1512.0    234.81  1.0  1782.0   
Low                      99.2      176.99  1.0  1506.0    260.01  1.0  5612.0   

                           scaf_bp                         contig_bp  \
                              mean        min        max        mean   
Classification_quality                                                 
High                    3798407.46  1378189.0  7005797.0  3796823.28   
Medium                  3494803.61  1043041.0  7041311.0  3485573.28   
Low                     2590496.53   569462.0  7026888.0  2576486.22   

                                             gap_pct             scaf_N50  \
                              min        max    mean  min    max     mean   
Classification_quality                                                      
High                    1378189.0  7005740.0    0.04  0.0   8.13     6.04   
Medium                  1043041.0  7041311.0    0.29  0.0   3.20    30.85   
Low                      569462.0  6570299.0    0.60  0.0  11.48    41.63   

                                      scaf_L50                     ctg_N50  \
                        min    max        mean      min        max    mean   
Classification_quality                                                       
High                    1.0   76.0  1131606.39  12138.0  6363829.0    6.73   
Medium                  1.0  353.0   552570.71   3040.0  6653812.0   39.04   
Low                     1.0  418.0   408083.12   2659.0  5602501.0   53.25   

                                        ctg_L50                     scaf_N90  \
                        min     max        mean      min        max     mean   
Classification_quality                                                         
High                    1.0    80.0  1035143.71  11189.0  6209424.0    18.60   
Medium                  1.0   370.0   522564.81   3040.0  6653812.0    99.93   
Low                     1.0  1103.0   392454.35   1884.0  5602501.0   123.54   

                                      scaf_L90                    ctg_N90  \
                        min     max       mean     min        max    mean   
Classification_quality                                                      
High                    1.0   253.0  866615.86  3054.0  6363829.0   20.92   
Medium                  1.0  1121.0  437711.30   545.0  6653812.0  128.07   
Low                     1.0  1182.0  314657.00  1241.0  5602501.0  162.97   

                                       ctg_L90                       scaf_max  \
                        min     max       mean     min        max        mean   
Classification_quality                                                          
High                    1.0 

In [54]:
with pd.ExcelWriter('Statistics.xlsx') as writer:  
    statistics.to_excel(writer, sheet_name="Statistics")
    statistics_classification.to_excel(writer, sheet_name="Statistics_per_quality")

-----------------------

## References

**CheckM** \
Parks DH, Imelfort M, Skennerton CT, Hugenholtz P, Tyson GW. 2015. CheckM: assessing the quality of microbial genomes recovered from isolates, single cells, and metagenomes. Genome Research, 25: 1043–1055. \
3rd party dependencies: \
pplacer: Matsen FA, Kodner RB, Armbrust EV. 2010. pplacer: linear time maximum-likelihood and Bayesian phylogenetic placement of sequences onto a fixed reference tree. BMC Bioinformatics 11: doi:10.1186/1471-2105-11-538. \
prodigal: Hyatt D, Locascio PF, Hauser LJ, Uberbacher EC. 2012. Gene and translation initiation site prediction in metagenomic sequences. Bioinformatics 28: 2223–2230. \
HMMER: http://hmmer.org/ \

**GTDB-Tk** \
Chaumeil PA, et al. 2019. GTDB-Tk: A toolkit to classify genomes with the Genome Taxonomy Database. Bioinformatics, btz848.\
3rd party dependencies: \
Matsen FA, et al. 2010. pplacer: linear time maximum-likelihood and Bayesian phylogenetic placement of sequences onto a fixed reference tree. BMC Bioinformatics, 11:538. \
Jain C, et al. 2019. High-throughput ANI Analysis of 90K Prokaryotic Genomes Reveals Clear Species Boundaries. Nat. Communications, doi: 10.1038/s41467-018-07641-9. \
Hyatt D, et al. 2010. Prodigal: prokaryotic gene recognition and translation initiation site identification. BMC Bioinformatics, 11:119. doi: 10.1186/1471-2105-11-119. \
Price MN, et al. 2010. FastTree 2 - Approximately Maximum-Likelihood Trees for Large Alignments. PLoS One, 5, e9490. \
Eddy SR. 2011. Accelerated profile HMM searches. PLOS Comp. Biol., 7:e1002195.
Ondov BD, et al. 2016. Mash: fast genome and metagenome distance estimation using MinHash. Genome Biol 17, 132. doi: doi: 10.1186/s13059-016-0997-x. 

**BBtools**
https://jgi.doe.gov/data-and-tools/bbtools/
Bushnell, B. BBMap short read aligner, and other bioinformatic tools. Available online: http://sourceforge.net/projects/bbmap 